In [5]:
import tensorflow as tf
import os
import numpy as np

In [13]:
model_name = 'resnet50'

In [14]:
model = tf.keras.models.load_model('/mnt/d/datasets/INBREAST/results/' + model_name + '.h5')

In [17]:
# Check CUDA functionality, restart kernel to change GPUs
gpus = tf.config.list_physical_devices('GPU')
print("*************************************")
print(gpus)
print("*************************************")

cutoff = 0.5
img_height = 224
img_width = 224
batch_size = 6

# setup train, validation, and test folders
traindir = '/mnt/d/datasets/INBREAST/split/train'
valdir = '/mnt/d/datasets/INBREAST/split/validation'
testdir = '/mnt/d/datasets/INBREAST/split/test'
dirName = '2_classes'

*************************************
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
*************************************


In [ ]:
test_ds = tf.keras.utils.image_dataset_from_directory(
  testdir,
  image_size=(img_height, img_width),
  batch_size=batch_size
)

In [18]:
# Define function to preprocess images as required by ResNet
def preprocess(images, labels):
    return tf.keras.applications.resnet_v2.preprocess_input(images), labels

test = tf.keras.preprocessing.image_dataset_from_directory(
    testdir, image_size=(img_width, img_height), batch_size=batch_size)

test_ds = test.map(preprocess)

Found 492 files belonging to 2 classes.


In [19]:
testloss, testaccuracy, testauc, precision, recall = model.evaluate(test_ds)
print('Test accuracy :', testaccuracy)
print('Test AUC :', testauc)

F1 = 2 * float(precision) * float(recall) / (float(precision) + float(recall))
print('Test F1 :', F1)
print('Test precision :', precision)
print('Test recall :', recall)

# Model path setup.
if not os.path.exists('/mnt/d/datasets/INBREAST/results/' + model_name + '/' + dirName):
    os.makedirs('/mnt/d/datasets/INBREAST/results/' + model_name + '/' + dirName + '/')

model.save('/mnt/d/datasets/INBREAST/results/' + model_name + '/' + dirName + '/custom_cnn_v_1')
predicted_probs = np.array([])
true_classes = np.array([])
IterationChecker = 0
for images, labels in test_ds:
    if IterationChecker == 0:
        predicted_probs = model(images)
        true_classes = labels.numpy()

    IterationChecker += 1

    predicted_probs = np.concatenate([predicted_probs,
                                      model(images)])
    true_classes = np.concatenate([true_classes, labels.numpy()])
# Since they are sigmoid outputs, you need to transform them into classes with a threshold, i.e 0.5 here:
predicted_classes = [1 * (x[0] >= cutoff) for x in predicted_probs]
# confusion matrix etc:
conf_matrix = tf.math.confusion_matrix(true_classes, predicted_classes)
print(conf_matrix)

predicted_probs = np.squeeze(predicted_probs)
predicted_classes = np.array(predicted_classes)
true_classes = np.squeeze(true_classes)
summedResults = np.stack((predicted_probs, predicted_classes, true_classes), axis=1)
##Print out statistics which test files are correctly predicted or not.
np.savetxt('/mnt/d/datasets/INBREAST/results/' + model_name + '.csv', summedResults, delimiter=',',
           header="predicted_probabilty,predicted_classes,true_classes", comments="")

82/82 [==============================] - 6s 70ms/step - loss: 0.8055 - accuracy: 0.8069 - auc: 0.8685 - precision: 0.8427 - recall: 0.8094
Test accuracy : 0.8069105744361877
Test AUC : 0.8684697151184082
Test F1 : 0.8256880623394667
Test precision : 0.8426966071128845
Test recall : 0.8093525171279907


INFO:tensorflow:Assets written to: /mnt/d/datasets/INBREAST/results/resnet50/2_classes/custom_cnn_v_1/assets


INFO:tensorflow:Assets written to: /mnt/d/datasets/INBREAST/results/resnet50/2_classes/custom_cnn_v_1/assets


tf.Tensor(
[[172  42]
 [ 55 229]], shape=(2, 2), dtype=int32)
